In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import datetime
import copy
import time

import sys
sys.path.append('../../')

from data.processing import get_data

import models
import random

from main.seir.fitting import single_fitting_cycle
from main.seir.forecast import get_forecast, forecast_all_trials, create_all_trials_csv, create_decile_csv_new
from main.seir.sensitivity import calculate_sensitivity_and_plot
import main.seir.uncertainty as uncertainty_module
from utils.generic.create_report import save_dict_and_create_report_simple
from utils.generic.config import read_config
from utils.generic.enums import Columns
from utils.fitting.loss import Loss_Calculator
from utils.generic.logging import log_wandb
from viz import plot_forecast, plot_top_k_trials, plot_ptiles
from main.seir.recovery import loss_parameter_recovery

import yaml

&lt;IPython.core.display.HTML object&gt;

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Wandb version 0.10.5 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [3]:
predictions_dict = {}

In [4]:
config_filename = 'default.yaml'
config = read_config(config_filename)

In [5]:
output_folder = '../../misc/reports/{}'.format(datetime.datetime.now().strftime("%Y_%m%d_%H%M%S"))

## Perform M1 and M2 fits

In [22]:
best_params = []
train_losses = []
val_losses = []
n, m = 2, 2
for j in range(m):
    r = random.randint(1,100)
    for i in range(n):
        print (j, i, r)
        np.random.seed(r)
        predictions_dict['m1'] = single_fitting_cycle(**copy.deepcopy(config['fitting'])) 
        predictions_dict['fitting_date'] = datetime.datetime.now().strftime("%Y-%m-%d")
        best_params.append(predictions_dict['m1']['best_params'])
        train_losses.append(predictions_dict['m1']['df_loss']['train'])
        val_losses.append(predictions_dict['m1']['df_loss']['val'])

AttributeError: module &#39;random&#39; has no attribute &#39;ranint&#39;

In [21]:
loss_parameter_recovery(predictions_dict['m1']['ideal_params'], predictions_dict['m1']['best_params'], loss_function='rmse')

{&#39;lockdown_R0&#39;: 0.047992680202913474,
 &#39;T_inc&#39;: 0.531371997903376,
 &#39;T_inf&#39;: 0.16626397753684907,
 &#39;T_recov&#39;: 1.6076987910173095,
 &#39;T_recov_fatal&#39;: 2.4951240331606357,
 &#39;P_fatal&#39;: 0.02519289776748962,
 &#39;E_hosp_ratio&#39;: 0.36372900219001214,
 &#39;I_hosp_ratio&#39;: 0.0009532273236694255}

In [22]:
import pandas as pd

df = pd.DataFrame(best_params)
df

,E_hosp_ratio,I_hosp_ratio,P_fatal,T_inc,T_inf,T_recov,T_recov_fatal,lockdown_R0
0,0.076528,0.31218,0.099847,4.380716,3.240771,29.888949,3.449618,1.02282


In [23]:
df2 = pd.DataFrame(train_losses)
df2 = df2.add_prefix('trainloss_')
df2 = df2.set_index(df.index)

df3 = pd.DataFrame(val_losses)
df3 = df3.add_prefix('valloss_')
df3 = df3.set_index(df.index)

df2

,trainloss_total,trainloss_active,trainloss_recovered,trainloss_deceased
0,0.762948,3.846705,0.511069,9.21334


In [24]:
df_final = df.join(df2)
df_final = df_final.join(df3)

In [ ]:
df_final.to_csv('../../misc/reports/recovered_params3.csv')